
Before you start:

We used the DCNv2 code from https://github.com/3846chs/DCNv2 instead of the original code in the `smoke/csrc` folder.
But this code only works with torch 1.7 and CUDA 11.1, so that's what we used in the Dockerfile of this project 

Seems to work: build a docker based on torch 1.7 and CUDA 11.1 ==> we worked with these versions in our Dockerfile.

First do:
`python setup.py build develop`
in a shell at the top-level of the project to compile the C code. 


In [1]:
import torch

from PIL import Image
import torchvision.transforms as transforms

from smoke.config import cfg
from smoke.engine import (
    default_argument_parser,
    default_setup
)
from smoke.utils.check_point import DetectronCheckpointer
from smoke.modeling.detector import build_detection_model


In [2]:
def setup(args):
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    default_setup(cfg, args)
    return cfg


In [3]:
def do_infer(args):
    cfg = setup(args)

    model = build_detection_model(cfg)
    model.training = False
    model.backbone.training = False
    model.heads.training = False
    device = torch.device(cfg.MODEL.DEVICE)
    model.to(device)

    checkpointer = DetectronCheckpointer(cfg, model, save_dir=cfg.OUTPUT_DIR)
    ckpt = cfg.MODEL.WEIGHT if args.ckpt is None else args.ckpt
    _ = checkpointer.load(ckpt, use_latest=args.ckpt is None)

    # Read a PIL image
    image = Image.open('data/kylle-pangan-unsplash.jpg')
    image = image.resize([640, 320]) # we probably don't need to make it this small, but the original size gives an OOM 
    # Define a transform to convert PIL image to a Torch tensor
    # transform = transforms.Compose([
    #     transforms.PILToTensor()
    # ])
    transform = transforms.PILToTensor()
    # Convert the PIL image to Torch tensor
    img_tensor = transform(image)

    img_tensor = img_tensor.to(device)
    output = model(img_tensor.float())
    cpu_device = torch.device("cpu")
    output = output.to(cpu_device)

    return output

In [4]:
args = default_argument_parser().parse_args(['--eval-only', '--config-file', 'configs/smoke_gn_vector.yaml'])
print("Command Line Args:", args)
output = do_infer(args)

output

Command Line Args: Namespace(ckpt=None, config_file='configs/smoke_gn_vector.yaml', dist_url='tcp://127.0.0.1:50152', eval_only=True, machine_rank=0, num_gpus=1, num_machines=1, opts=[])
[2022-12-05 08:16:44,927] smoke INFO: Using 1 GPUs
[2022-12-05 08:16:44,929] smoke INFO: Collecting environment info
[2022-12-05 08:16:46,556] smoke INFO: 
PyTorch version: 1.7.0
Is debug build: True
CUDA used to build PyTorch: 11.0
ROCM used to build PyTorch: N/A

OS: Ubuntu 18.04.5 LTS (x86_64)
GCC version: (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Clang version: Could not collect
CMake version: Could not collect

Python version: 3.8 (64-bit runtime)
Is CUDA available: True
CUDA runtime version: Could not collect
GPU models and configuration: GPU 0: NVIDIA GeForce RTX 3090
Nvidia driver version: 510.85.02
cuDNN version: Probably one of the following:
/usr/lib/x86_64-linux-gnu/libcudnn.so.8.0.4
/usr/lib/x86_64-linux-gnu/libcudnn_adv_infer.so.8.0.4
/usr/lib/x86_64-linux-gnu/libcudnn_adv_train.so.8.0.4
/usr/l

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/functional.py:127: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  img = torch.as_tensor(np.asarray(pic))


- - - - - - - - -
projected points, size torch.Size([50, 2]):
tensor([[ 41.0000,  74.2562],
        [123.0000,  72.7688],
        [ 98.0000,  75.6125],
        [ 76.0000,  73.4750],
        [ 88.0000,  71.5500],
        [101.0000,  74.6312],
        [ 96.0000,  78.6000],
        [ 62.0000,  73.3875],
        [ 69.0000,  20.4312],
        [135.0000,   3.8438],
        [ 66.0000,  19.4125],
        [ 55.0000,  49.3438],
        [133.0000,   4.8313],
        [  4.0000,  59.0250],
        [153.0000,   3.9562],
        [ 95.0000,  75.5938],
        [  5.0000,   4.0312],
        [ 69.0000,  25.4312],
        [  8.0000,   4.0500],
        [ 95.0000,  57.5938],
        [125.0000,  74.7812],
        [ 57.0000,  49.3563],
        [ 70.0000,  18.4375],
        [ 70.0000,  23.4375],
        [ 50.0000,  60.3125],
        [ 87.0000,  78.5438],
        [138.0000,   5.8625],
        [ 19.0000,   2.1188],
        [108.0000,  76.6750],
        [122.0000,   4.7625],
        [ 76.0000,  78.4750],
        

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [7]:
torch.set_printoptions(profile="full")
print(output)

tensor([], size=(0, 9), grad_fn=<CopyBackwards>)
